<a href="https://colab.research.google.com/github/anna4j/Movie-Recommendation-API/blob/master/Jupyter%20Notebook/recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
movies = 'https://github.com/anna4j/Movie-Recommendation-API/blob/master/Raw%20Dataset/TMDB%205000%20Movie%20Dataset/tmdb_5000_movies.csv?raw=true'
data_movies = pd.read_csv(movies ,na_filter=False)

credits = 'https://github.com/anna4j/Movie-Recommendation-API/blob/master/Raw%20Dataset/TMDB%205000%20Movie%20Dataset/tmdb_5000_credits.csv?raw=true'
data_credits = pd.read_csv(credits)

In [3]:
data_credits.columns = ['movie_id','tittle','cast','crew']
data_movies = data_movies.merge(data_credits,on='movie_id')

In [4]:
data_movies.head(3)


,budget,genres,homepage,movie_id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,tittle,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",10-12-2009,2787965087,162,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",19-05-2007,961000000,169,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",26-10-2015,880674609,148,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


In [5]:
data_movies = data_movies.drop(columns=['tittle', 'tagline', 'status', 'homepage', 
                                        'keywords','crew','vote_count', 'vote_average',
                                       'tagline', 'spoken_languages', 'runtime',
                                       'popularity', 'production_companies', 'budget',
                                       'production_countries', 'release_date', 'revenue',
                                        'title', 'original_language'])

data_movies.head(2)

,genres,movie_id,original_title,overview,cast
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa..."


In [6]:
features = ['cast', 'genres']
for feature in features:
    data_movies[feature] = data_movies[feature].apply(literal_eval)

In [7]:
def get_list(meta_data):
    if isinstance(meta_data, list):
        names = [col['name'] for col in meta_data]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []

features = ['cast','genres']
for feature in features:
    data_movies[feature] = data_movies[feature].apply(get_list)

In [8]:
data_movies.head(2)


,genres,movie_id,original_title,overview,cast
0,"[Action, Adventure, Fantasy]",19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]"
1,"[Adventure, Fantasy, Action]",285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Johnny Depp, Orlando Bloom, Keira Knightley]"


In [9]:
#data_movies['cast'] = data_movies['cast'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x if len(name)>3])
#data_movies['genres'] = data_movies['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x if len(name)>3])

#data_movies.head(2)

In [10]:
hindi_movies = 'https://github.com/anna4j/Movie-Recommendation-API/blob/master/Raw%20Dataset/The%20Indian%20Movie%20Database/Final/bollywood_full_1950-2019.csv?raw=true'
data_movies_hindi = pd.read_csv(hindi_movies, na_filter=False)

data_movies_hindi.head(2)

,title_x,imdb_id,poster_path,wiki_link,title_y,original_title,is_adult,year_of_release,runtime,genres,imdb_rating,imdb_votes,story,summary,tagline,actors,wins_nominations,release_date
0,Uri: The Surgical Strike,tt8291224,https://upload.wikimedia.org/wikipedia/en/thum...,https://en.wikipedia.org/wiki/Uri:_The_Surgica...,Uri: The Surgical Strike,Uri: The Surgical Strike,0,2019,138,Action|Drama|War,8.4,35112.0,Divided over five chapters the film chronicle...,Indian army special forces execute a covert op...,,Vicky Kaushal|Paresh Rawal|Mohit Raina|Yami Ga...,4 wins,11 January 2019 (USA)
1,Battalion 609,tt9472208,,https://en.wikipedia.org/wiki/Battalion_609,Battalion 609,Battalion 609,0,2019,131,War,4.1,73.0,The story revolves around a cricket match betw...,The story of Battalion 609 revolves around a c...,,Vicky Ahuja|Shoaib Ibrahim|Shrikant Kamat|Elen...,,11 January 2019 (India)


In [11]:
data_movies_hindi = data_movies_hindi.drop(columns=['title_x', 'tagline', 'poster_path', 
                                        'wiki_link','title_y','is_adult', 'year_of_release',
                                       'tagline', 'imdb_rating', 'runtime',
                                       'imdb_votes', 'story',
                                       'wins_nominations', 'release_date'])

data_movies_hindi.rename(columns = {'actors':'cast', 'imdb_id':'movie_id'}, inplace = True)

data_movies_hindi['genres'] = data_movies_hindi['genres'].str.split('|')

data_movies_hindi['cast'] = data_movies_hindi['cast'].str.split('|')

data_movies_hindi = data_movies_hindi.drop([697])

data_movies_hindi.head()

,movie_id,original_title,genres,summary,cast
0,tt8291224,Uri: The Surgical Strike,"[Action, Drama, War]",Indian army special forces execute a covert op...,"[Vicky Kaushal, Paresh Rawal, Mohit Raina, Yam..."
1,tt9472208,Battalion 609,[War],The story of Battalion 609 revolves around a c...,"[Vicky Ahuja, Shoaib Ibrahim, Shrikant Kamat, ..."
2,tt6986710,The Accidental Prime Minister,"[Biography, Drama]",Explores Manmohan Singh's tenure as the Prime ...,"[Anupam Kher, Akshaye Khanna, Aahana Kumra, At..."
3,tt8108208,Why Cheat India,"[Crime, Drama]",The movie focuses on existing malpractices in ...,"[Emraan Hashmi, Shreya Dhanwanthary, Snighdade..."
4,tt6028796,Evening Shadows,[Drama],Under the 'Evening Shadows' truth often plays...,"[Mona Ambegaonkar, Ananth Narayan Mahadevan, D..."


In [12]:
def clean_data(actors):
    if isinstance(actors, list):
        return actors[:3]

features = ['cast']
for feature in features:
    data_movies_hindi[feature] = data_movies_hindi[feature].apply(clean_data)

data_movies_hindi.head()

,movie_id,original_title,genres,summary,cast
0,tt8291224,Uri: The Surgical Strike,"[Action, Drama, War]",Indian army special forces execute a covert op...,"[Vicky Kaushal, Paresh Rawal, Mohit Raina]"
1,tt9472208,Battalion 609,[War],The story of Battalion 609 revolves around a c...,"[Vicky Ahuja, Shoaib Ibrahim, Shrikant Kamat]"
2,tt6986710,The Accidental Prime Minister,"[Biography, Drama]",Explores Manmohan Singh's tenure as the Prime ...,"[Anupam Kher, Akshaye Khanna, Aahana Kumra]"
3,tt8108208,Why Cheat India,"[Crime, Drama]",The movie focuses on existing malpractices in ...,"[Emraan Hashmi, Shreya Dhanwanthary, Snighdade..."
4,tt6028796,Evening Shadows,[Drama],Under the 'Evening Shadows' truth often plays...,"[Mona Ambegaonkar, Ananth Narayan Mahadevan, D..."


In [13]:
data_movies_hindi.isnull().sum()


movie_id          0
original_title    0
genres            0
summary           0
cast              0
dtype: int64

In [14]:
combine = data_movies.append(data_movies_hindi, ignore_index=True, sort=True)

combine = combine.drop([5160])

#combine['plot'] = combine[['overview', 'summary']].apply(lambda x: ' '.join(x), axis = 1) 

#combine = combine.drop(['summary','overview'],axis=1)

combine  #5160

,cast,genres,movie_id,original_title,overview,summary
0,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[Action, Adventure, Fantasy]",19995,Avatar,"In the 22nd century, a paraplegic Marine is di...",NaN
1,"[Johnny Depp, Orlando Bloom, Keira Knightley]","[Adventure, Fantasy, Action]",285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",NaN
2,"[Daniel Craig, Christoph Waltz, Léa Seydoux]","[Action, Adventure, Crime]",206647,Spectre,A cryptic message from Bond’s past sends him o...,NaN
3,"[Christian Bale, Michael Caine, Gary Oldman]","[Action, Crime, Drama]",49026,The Dark Knight Rises,Following the death of District Attorney Harve...,NaN
4,"[Taylor Kitsch, Lynn Collins, Samantha Morton]","[Action, Adventure, Science Fiction]",49529,John Carter,"John Carter is a war-weary, former military ca...",NaN
...,...,...,...,...,...,...
9127,"[Ashok Kumar, Nalini Jaywant, Kuldip Kaur]",[Drama],tt0268614,Samadhi,NaN,The story is based on the true incident at INA...
9128,"[Ashok Kumar, Nalini Jaywant, Nawab]",[Drama],tt0244182,Sangram,NaN,After the death of his wife a policeman fails...
9129,"[Raj Kapoor, Rehana, Om Prakash]","[Drama, Family]",tt0269826,Sargam,NaN,Add a Plot »
9130,"[Sohrab Modi, Naseem Banu, Pushpa Hans]",[Drama],tt0243555,Sheesh Mahal,NaN,Thakur Jaspal Singh lives in the prestigious a...


In [15]:
combine.to_csv('movie_data_test.csv', index=False)


In [16]:
pd.set_option('display.max_rows', None)

final_data = pd.read_csv('movie_data_test.csv', na_filter=False)

final_data['plot'] = final_data[['overview', 'summary']].apply(lambda x: ' '.join(x), axis = 1) 

final_data = final_data.drop(['summary','overview'],axis=1)

final_data = final_data[:-1]

In [17]:
final_data.head(5)

,cast,genres,movie_id,original_title,plot
0,"['Sam Worthington', 'Zoe Saldana', 'Sigourney ...","['Action', 'Adventure', 'Fantasy']",19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,"['Johnny Depp', 'Orlando Bloom', 'Keira Knight...","['Adventure', 'Fantasy', 'Action']",285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,"['Daniel Craig', 'Christoph Waltz', 'Léa Seydo...","['Action', 'Adventure', 'Crime']",206647,Spectre,A cryptic message from Bond’s past sends him o...
3,"['Christian Bale', 'Michael Caine', 'Gary Oldm...","['Action', 'Crime', 'Drama']",49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,"['Taylor Kitsch', 'Lynn Collins', 'Samantha Mo...","['Action', 'Adventure', 'Science Fiction']",49529,John Carter,"John Carter is a war-weary, former military ca..."


In [18]:
final_data.to_csv('movie_data_final.csv', index = False)

In [19]:
from google.colab import files
files.download('movie_data_final.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>